# Oval Annotation (COCO Classes)

Script for drawing bounding boxes around relevant objects. 2 different models have been implemented in this script, FasterRCNN and RetinaNet. Choose which ever is appropriate (or use both model detections together for better accuracy!)

The following objects will be detected with this script: 'airplane', 'fire hydrant', 'street sign', 'bench', 'bicycle', 'chair', 'dining table'

In [5]:
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
from torchvision.models.detection import retinanet_resnet50_fpn_v2, RetinaNet_ResNet50_FPN_V2_Weights

# Map COCO class indices to your selected classes
coco_to_selected = {
    5: 'airplane',
    11: 'fire hydrant',
    12: 'street sign',
    15: 'bench',
    2: 'bicycle',
    62: 'chair',
    67: 'dining table'
}

# Use below variables in successive cells if you want to use FasterRCNN model
frcnn_weights = FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT
frcnn_preprocess = frcnn_weights.transforms()
frcnn_model = fasterrcnn_mobilenet_v3_large_fpn(weights=frcnn_weights, progress=False)

# RetinaNet model
r_weights = RetinaNet_ResNet50_FPN_V2_Weights.DEFAULT
r_preprocess = r_weights.transforms()
r_model = retinanet_resnet50_fpn_v2(weights=r_weights, progress=False)



# change to correct model here
model = r_model.eval()

In [6]:
import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
import os
import shutil

def run_inference(model, image_path, output_folder, confidence_threshold=0.5):

    # Check if GPU is available and move model to GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    image = Image.open(image_path).convert("RGB")
    p_image = r_preprocess(image) # apply model specific pre-process from above cell
    image_tensor = p_image.unsqueeze(0)

    model.eval()

    with torch.no_grad():
        predictions = model(image_tensor)

    # Interpret the results
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    
    # Filter predictions based on confidence threshold
    mask = [(score > confidence_threshold and label.item() in coco_to_selected) 
                     for score, label in zip(scores, labels)] 
    boxes = boxes[mask]
    scores = scores[mask]
    labels = labels[mask]


    # Prepare data for JSON
    results = []
    for box, label, score in zip(boxes, labels, scores):
        results.append({
            'box': box.tolist(),
            'label': coco_to_selected.get(int(label), 'Unknown'),
            'score': float(score)
    })
        
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Save bounding box data to JSON file in the output folder
    base_name = os.path.basename(image_path)
    json_path = os.path.join(output_folder, os.path.splitext(base_name)[0] + ".json")
    with open(json_path, 'w') as f:
        json.dump(results, f, indent=4)




    # # Visualize results
    # fig, ax = plt.subplots(1)
    # ax.imshow(image)

    # for box, label, score in zip(boxes, labels, scores):
    #     rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], 
    #                              linewidth=1, edgecolor='r', facecolor='none')
    #     ax.add_patch(rect)
    #     ax.text(box[0], box[1], f"{label}: {score:.2f}", 
    #             bbox=dict(facecolor='white', alpha=0.8))

    # plt.show()



In [8]:
# Run for multiple images

import glob

# Run inference
image_folder_path = "C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4/*.jpg"
# Define the output folder for JSON files
output_folder = "C:/Users/yujim/Documents/Project_Oval/images/output_json"


img_paths = sorted(glob.glob(image_folder_path))

# Override the output folder if it already exists
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder, exist_ok=True)

print(img_paths)

for img in img_paths:
    run_inference(model, img, output_folder, confidence_threshold=0.5)


# Single image inference

# img = "./oval_samples/08_57_43.jpg"
# run_inference(model, img, confidence_threshold=0.5)


['C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_48.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_48_darker.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_48_darkest.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_51.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_51_darker.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_51_darkest.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_54.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_54_darker.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_54_darkest.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_57.jpg', 'C:/Users/yujim/Documents/Project_Oval/images/captured_images_test4\\08_54_57_darker.jpg', 'C:/Users/yujim/Documen